In [ ]:
import numpy as np
import cv2
from tqdm import tqdm
from pathlib import Path
from gtsam.symbol_shorthand import X
import matplotlib.pyplot as plt

from lac.slam.feature_tracker import FeatureTracker
from lac.perception.segmentation import UnetSegmentation
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.util import load_data, load_stereo_images
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)

# Load the ground truth map
map = np.load(
    Path(LAC_BASE_PATH) / "data/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
segmentation = UnetSegmentation()
tracker = FeatureTracker(cam_config)

Frontend:

- Run both segmentation and feature extraction on left and right images
- Triangulate feature matches. For keypoints labeled as rock, group them together

Rock map:

- Each rock has a set of world points with associated descriptors (these descriptors should probably also be associated with a viewing direction since the apperance of a rock can change with viewing direction)
-

Graph SLAM:

- In the graph, we have a landmark for each rock corresponding to its centroid point
- For each keyframe, we determine the observed pixel centroid of the rock based on segementation outputs, and use that to add reprojection factors


In [ ]:
START_FRAME = 80

In [ ]:
left_image = left_imgs[100]
right_image = right_imgs[100]

fig, ax = plt.subplots(1, 2, figsize=(18, 10))
ax[0].imshow(left_image, cmap="gray")
ax[1].imshow(right_image, cmap="gray")
ax[0].axis("off")
ax[1].axis("off")
plt.subplots_adjust(wspace=0.03)
plt.show()

In [ ]:
from lac.utils.visualization import overlay_mask

In [ ]:
left_masks, left_full_mask = segmentation.segment_rocks(left_image)
right_masks, right_full_mask = segmentation.segment_rocks(right_image)

left_seg_overlay = overlay_mask(left_image, left_full_mask, color=(1, 0, 0))
right_seg_overlay = overlay_mask(right_image, right_full_mask, color=(1, 0, 0))

fig, ax = plt.subplots(1, 2, figsize=(18, 10))
ax[0].imshow(left_seg_overlay)
ax[1].imshow(right_seg_overlay)
ax[0].axis("off")
ax[1].axis("off")
plt.subplots_adjust(wspace=0.03)
plt.show()

In [ ]:
from lac.slam.feature_tracker import prune_features

In [ ]:
left_feats, right_feats, matches, depths = tracker.process_stereo(left_image, right_image)

left_matched_feats = prune_features(left_feats, matches[:, 0])
left_matched_pts = left_matched_feats["keypoints"][0].cpu().numpy()
right_matched_feats = prune_features(right_feats, matches[:, 1])
right_matched_pts = right_matched_feats["keypoints"][0].cpu().numpy()

In [ ]:
# Filter to points that are within segmentations

# Dilate the masks
kernel = np.ones((5, 5), np.uint8)
left_full_mask_dilated = cv2.dilate(left_full_mask, kernel, iterations=1)
right_full_mask_dilated = cv2.dilate(right_full_mask, kernel, iterations=1)

rock_pt_idxs = []

for i in range(len(left_matched_pts)):
    x_left, y_left = left_matched_pts[i]
    x_right, y_right = right_matched_pts[i]
    if (
        left_full_mask_dilated[int(y_left), int(x_left)]
        and right_full_mask_dilated[int(y_right), int(x_right)]
    ):
        rock_pt_idxs.append(i)

In [ ]:
left_rock_matched_pts = left_matched_pts[rock_pt_idxs]
right_rock_matched_pts = right_matched_pts[rock_pt_idxs]

In [ ]:
from lightglue import viz2d

viz2d.plot_images([left_seg_overlay, right_seg_overlay])
viz2d.plot_matches(left_rock_matched_pts, right_rock_matched_pts, color="lime", lw=0.2)